### Recommendation system
Generate recommendations using specific examples

In [114]:
from pymongo import MongoClient
from pprint import pprint

import pandas as pd
import re
import nltk

import pickle
import json

In [115]:
# import product list
with open('pickles/product_list.pickle', 'rb') as to_read:
    product_list = pickle.load(to_read)

with open('pickles/review_list.pickle', 'rb') as to_read:
    review_list = pickle.load(to_read)
    
with open('pickles/sim_asin.pickle', 'rb') as to_read:
    sim_asin = pickle.load(to_read)

with open('pickles/all_review_list_SA.pickle', 'rb') as to_read:
    all_review_list = pickle.load(to_read)

In [116]:
all_review_list.head(5)

,asin,title,all_review_text,sentences,compound_scores,pos_sents,pos_scores,neg_sents,neg_scores,topic_3_sent,...,topic_3_score,topic_4_sent,topic_4_score1,topic_4_score,topic_5_sent,topic_5_score1,topic_5_score,topic_6_sent,topic_6_score1,topic_6_score
0,B0000Y3NO6,"DERMAdoctor Calm, Cool & Corrected anti-rednes...",I gave it a shot for a while until the small b...,[I gave it a shot for a while until the small ...,"[-0.2716, 0.3716, 0.6369, -0.6597, 0.1621, 0.7...","[The cream feels cool on the skin, which is ni...","[0.3716, 0.6369, 0.1621, 0.7506, 0.8242, 0.510...",[I gave it a shot for a while until the small ...,"[-0.2716, -0.6597, -0.6358, -0.2365, 0.0, 0.0,...",[This extremely soothing cream helps any irrit...,...,0.334000,"[It also dimished my acne outbreaks, which I t...","[0.2023, 0.6306, 0.0516, 0.6588, 0.4767, 0.419...",0.436043,[After trying all the drugstore rosacea produc...,"[0.6369, 0.8242, 0.5106, 0.1406, 0.4939, 0.624...",0.577633,"[It also dimished my acne outbreaks, which I t...",[0.2023],0.20230
1,B00012C5RS,"DERMAdoctor Picture Porefect Pore Minimizer, 1...","I haven't splurged on a full-size bottle yet, ...","[I haven't splurged on a full-size bottle yet,...","[0.3612, 0.0, 0.5994, 0.0, 0.0, 0.0, 0.4939, 0...","[I haven't splurged on a full-size bottle yet,...","[0.3612, 0.5994, 0.4939, 0.3384, 0.646, 0.3182...","[I received a sample of this, so the effects I...","[0.0, 0.0, 0.0, 0.0, 0.0, -0.5267, 0.0, -0.153...","[This product really does work, just beware if...",...,0.318200,[My skin is normally pretty oily and prone to ...,"[0.4939, 0.3384, 0.1027, 0.4201, 0.3089, 0.799...",0.359314,[My skin is normally pretty oily and prone to ...,"[0.4939, 0.3182, 0.0276, 0.4201, 0.4404]",0.340040,[I rant yet again: the packaging: Pretty lav...,"[0.2023, 0.8885]",0.54540
2,B0001EKTTC,"Glytone Rejuvenating Mask, 3 oz.",Great product . Gets the job done. Leaves your...,"[Great product ., Gets the job done., Leaves y...","[0.6249, 0.0, 0.4173, 0.6194, 0.8555, 0.5574, ...","[Great product ., Leaves your skin absolutely ...","[0.6249, 0.4173, 0.6194, 0.8555, 0.5574, 0.722...","[Gets the job done., This is a keeper!!!!!.., ...","[0.0, 0.0, -0.5267, 0.0, -0.296, 0.0, -0.34, -...",[],...,NaN,"[Perfect for acne prone skin., I use it once a...","[0.5719, 0.6486, 0.886, 0.2732]",0.594925,"[Very, very good deep cleansing masque, for oi...","[0.6194, 0.4201, 0.886, 0.7644, 0.8176]",0.701500,[],[],NaN
3,B0001EL5Q8,"PCA SKIN Rejuvenating Serum, 1 fl. oz.",I was very happy with the promptness with whic...,[I was very happy with the promptness with whi...,"[0.8479, 0.0, -0.2481, 0.1511, 0.8934, 0.0, 0....",[I was very happy with the promptness with whi...,"[0.8479, 0.1511, 0.8934, 0.5859, 0.5719, 0.790...",[Time will tell as to whether or not a 70 year...,"[0.0, -0.2481, 0.0, -0.4019, 0.0, 0.0, -0.4019...",[Reduces the signs of aging love itFast delive...,...,0.674000,[As soon as I stop using the serums my acne co...,[0.7227],0.722700,"[love itLove the way it feels, but not sure if...","[0.9703, 0.3182]",0.644250,[I really like the Pca line and notice the dif...,"[0.4201, 0.4754, 0.7964, 0.7506, 0.2023]",0.52896
4,B0001EL5JA,PCA SKIN Protecting Hydrator Broad Spectrum S...,I bought the Hydrator with SPF after having a ...,[I bought the Hydrator with SPF after having a...,"[0.6249, 0.0, 0.3612, 0.8114, 0.6696, 0.1027, ...",[I bought the Hydrator with SPF after having a...,"[0.6249, 0.3612, 0.8114, 0.6696, 0.1027, 0.440...",[I thought the Hydrator would be a similar tex...,"[0.0, 0.0, -0.2755, 0.0, -0.4003, 0.0, 0.0, -0...",[Highly recommended for sensitive and acne-pro...,...,0.529167,[Best SPF day moisturizer for oily/acne-prone ...,"[0.6696, 0.2716, 0.802]",0.581067,[Best SPF day moisturizer for oily/acne-prone ...,"[0.6696, 0.6249, 0.8457, 0.8561, 0.2682, 0.456...",0.631994,[],[],NaN


In [117]:
# recommend:
# - top 5 products for each category
# - return dictionary with the product info

In [118]:
sim = pd.DataFrame(columns=['prod1','prod2','similarity'])

sim['prod1'] = [i[1][0] for i in sim_asin]
sim['prod2'] = [i[1][1] for i in sim_asin]
sim['similarity'] = [i[0][0][0] for i in sim_asin]

In [119]:
sim.tail(5)

,prod1,prod2,similarity
2509915,B0000Y3NO6,B00AF3RYHE,0.0
2509916,B0000Y3NO6,B008KAA8P4,0.0
2509917,B0000Y3NO6,B005JEK7DC,0.0
2509918,B0000Y3NO6,B003VCKZZO,0.0
2509919,B0000Y3NO6,B000AV31NC,0.0


### generate recommendation

In [120]:
concerns = ['sensitive_skin','acne_control','anti-aging','oil_control','none']

In [121]:
# generate top 5 matches in each category

def get_rec(asin,concern):
    
    def get_other_prod(x):
        if x['prod1'] != asin:
            return x['prod1']
        else:
            return x['prod2']
    
    def get_sims(asin):
        df = pd.DataFrame(sim[(sim['prod1']==asin)|(sim['prod2']==asin)])
        df['selected_asin'] = asin
        df['other_asin'] = df.apply(get_other_prod, axis=1)
        df.drop(['prod1','prod2'],axis=1,inplace=True)
        return df

    # get similarities for that product
    prod = get_sims(asin)
    
    #merge the concern sentiments
    prod2 = pd.merge(prod,all_review_list,left_on='other_asin',right_on='asin',how='inner')
    
    # if concern is selected, sum that score with cosine similarity to amplify concern sentiment
    if concern == 'sensitive_skin':
        prod2['new_sim'] = round(prod2['similarity']*prod2['topic_3_score'],3)
#         prod2['new_sim'] = prod2['topic_3_score']
    elif concern == 'acne_control':
        prod2['new_sim'] = round(prod2['similarity']*prod2['topic_4_score'],3)
#         prod2['new_sim'] = prod2['topic_4_score']
    elif concern == 'oil_control':
        prod2['new_sim'] = round(prod2['similarity']*prod2['topic_5_score'],3)
#         prod2['new_sim'] = prod2['topic_5_score']
    elif concern == 'anti-aging':
        prod2['new_sim'] = round(prod2['similarity']*prod2['topic_6_score'],3)
#         prod2['new_sim'] = prod2['topic_6_score']
    elif concern == 'none':
        prod2['new_sim'] = round(prod2['similarity'],3)
    
    # join other product info
    prod3 = pd.merge(prod2,product_list,left_on='other_asin',right_on='asin',how='inner')
    
    main_prod_info = product_list[product_list['asin']==asin]
    
    prods = prod3.sort_values(['category2','new_sim'],ascending=False)
#     recs=prods
    recs = prods.groupby(['category2']).head(5)
    
    main_prod_info.to_csv('results/'+asin+'_'+concern+'_main.csv', sep = ",")
    recs[['other_asin','product','category2','overall_rating','similarity','new_sim','topic_3_score','topic_4_score','topic_5_score','topic_6_score']].to_csv('results/'+asin+'_'+concern+'_recs.csv', sep = ",")
    return main_prod_info, recs
#     return prod3

In [122]:
# test = get_rec('B0001EL5Q8') #anti-aging
# test = get_rec('B0001EL5JA') #hydrating
# main_prod, recs = get_rec('B0001EL5Q8','none')
# main_prod, recs = get_rec('B0001EL5Q8','anti-aging')
main_prod, recs = get_rec('B000UFWQJI','none')
main_prod2, recs2 = get_rec('B000UFWQJI','sensitive_skin')
print(main_prod.shape)
print(recs.shape)

(1, 8)
(35, 33)


In [123]:
main_prod

,id,asin,product,overall_rating,review_count,description,category2,category3
236,5dc881769f9b98109203c25a,B000UFWQJI,Epicuren Discovery Gelle Cleanser,4.5,11,"A deep cleansing, foaming gel that effectively...",Cleansers,Gels


#### check with no concerns

In [124]:
for _ in recs.loc[recs['other_asin']=='B000UFWQJI']['pos_sents']:
    print(_)

In [125]:
for _ in recs.loc[recs['other_asin']=='B000UFWQJI']['topic_3_sent']:
    print(_)

In [126]:
recs[['other_asin','product','category2','overall_rating','similarity','new_sim']]

,other_asin,product,category2,overall_rating,similarity,new_sim
0,B008H6MSZO,"Erno Laszlo Transphuse Age Defying Mask, 3.3 f...",Treatments & Masks,5.0,0.990278,0.990
4,B0177RLDYU,"THALGO Mceutic Pro-Renewal Cleanser, 6.76 oz",Treatments & Masks,5.0,0.956414,0.956
55,B00C1SZHBG,"DR. ALKAITIS Organic Flower Mask, 1.25 fl. oz.",Treatments & Masks,5.0,0.884563,0.885
56,B00HYLOVLW,"Pevonia Micro Retinol Foaming Cleanser, 4 Fl Oz",Treatments & Masks,3.8,0.881868,0.882
102,B00LNCB4A4,Borghese Fango Purificante Purifying Clay Clea...,Treatments & Masks,4.0,0.854337,0.854
131,B0019V5HI4,"Pevonia Combination Skin Cleanser, 6.8 Fl Oz",Toners & Astringents,3.6,0.831555,0.832
152,B01HBS7XP8,COSMEDIX Benefit Balance Antioxidant Infused T...,Toners & Astringents,5.0,0.811307,0.811
164,B00T72SWZM,"COSMEDIX Purity Solution, 3.3 Fluid Ounce",Toners & Astringents,5.0,0.799324,0.799
223,B002Q7K820,"Pevonia Spateen Blemished Skin Toner, 4 Fl Oz",Toners & Astringents,4.3,0.734536,0.735
241,B0018QXRCI,"Elemis Cleanser, Rehydrating Ginseng Toner For...",Toners & Astringents,5.0,0.715760,0.716


#### specify sensitive skin concern

In [127]:
recs2[['other_asin','product','category2','overall_rating','similarity','new_sim','topic_3_score','topic_4_score']]

,other_asin,product,category2,overall_rating,similarity,new_sim,topic_3_score,topic_4_score
179,B002RCQJ2W,Red Flower Lymphatic Phytopower Sea Cleanser a...,Treatments & Masks,4.4,0.788514,0.623,0.790600,0.176100
234,B00260GSDK,"DDF Glycolic 5% Exfoliating Wash, 8.45 fl. oz.",Treatments & Masks,3.1,0.719788,0.473,0.657500,0.468700
503,B011MFZM1I,IMAGE Skincare Ageless Total Resurfacing Masqu...,Treatments & Masks,4.2,0.505508,0.415,0.820100,0.728500
499,B00ZAA7O86,Laboratoires Filorga Hydra-Hyal Intensive Hydr...,Treatments & Masks,4.3,0.508875,0.412,0.809450,NaN
540,B0057J2M9Q,"Natura Bisse Tolerance Recovery Cream, 1.7 fl....",Treatments & Masks,4.4,0.489045,0.397,0.812600,NaN
223,B002Q7K820,"Pevonia Spateen Blemished Skin Toner, 4 Fl Oz",Toners & Astringents,4.3,0.734536,0.544,0.740300,0.601700
398,B00H59AA2W,"Guinot Hydra Beaute Toning Lotion, 6.7 Fl oz",Toners & Astringents,4.8,0.560834,0.510,0.910000,NaN
131,B0019V5HI4,"Pevonia Combination Skin Cleanser, 6.8 Fl Oz",Toners & Astringents,3.6,0.831555,0.451,0.541967,0.486833
395,B00172DJ7Q,Mario Badescu Special Cleansing Lotion C,Toners & Astringents,3.8,0.562556,0.410,0.728271,0.580226
445,B000N2KJAG,Erno Laszlo Value Size Light Controlling Lotio...,Toners & Astringents,3.7,0.533711,0.365,0.684200,NaN


#### compare top rated in that category for each

In [128]:
#no concerns - no sensitive skin references
samecat_none_asin = 'B000CR4GAQ'
for _ in recs.loc[recs['other_asin']==samecat_none_asin]['description']:
    print(_)

For combination, dry, mature and environmentally stressed skin types. June Jacobs, together with her daughter Rochelle, created this luxury skin care line centered on highly-efficacious, natural ingredients formulated to deliver immediate, impactful results with long-term benefits.


In [129]:
all_review_list[all_review_list['asin']==samecat_none_asin]['topic_3_sent']

100    []
Name: topic_3_sent, dtype: object

In [132]:
#sensitive skin - very positive sentiment
samecat_conc_asin = 'B00K1LYKUO'
for _ in recs2.loc[recs2['other_asin']==samecat_conc_asin]['description']:
    print(_)

This luxurious cleanser is powered by advanced peptides and plant-derived stem cell technology. It supports collagen, promotes elasticity and reduces the look of fines line and wrinkles. Carefully selected botanicals nourish the skin and guard against the daily onslaught of environmental pollutants. A revolutionary blend of apple, edelweiss and summer snowflake flower stem cells targets visible signs of aging, defends against environmental stress and smoothest wrinkles. Powerful peptides reduce the appearance of fine lines, wrinkles and sagging skin. The MAX stem cell facial cleanser is free of sulfates to promote a healthy skin balance, so skin looks smooth and glowing. IMAGE Skincare is a clean and professional line backed by clinical results. Created by an esthetician, formulated by a plastic surgeon, IMAGE is dedicated to helping its customers Age Later.


In [133]:
for _ in all_review_list[all_review_list['asin']==samecat_conc_asin]['topic_3_sent']:
    print(_)

["Thank you!Fantastic - thank you!Love the way this smells, and it's great for my overly sensitive skin."]


#### compare top rated in other categories

In [138]:
#no concerns - mask
# diffcat_none_asin = 'B008H6MSZO' #mask
diffcat_none_asin = 'B00260GSWQ' #exfoliator
# diffcat_none_asin = 'B01HBS7XP8' #toner
for _ in recs.loc[recs['other_asin']==diffcat_none_asin]['description']:
    print(_)

Advanced cleanser formula contains a breakthrough Turmeric Complex with exfoliating rice bran to accelerate surface cell turnover and eliminate impurities for youthful looking skin. By DDF


In [139]:
for _ in all_review_list[all_review_list['asin']==diffcat_none_asin]['topic_3_sent']:
    print(_)

[]


In [140]:
#sensitive skin - 
# diffcat_conc_asin = 'B002RCQJ2W' #mask
diffcat_conc_asin = 'B019QSK1YG' #exfoliator
# diffcat_conc_asin = 'B002Q7K820' #toner
for _ in recs2.loc[recs2['other_asin']==diffcat_conc_asin]['description']:
    print(_)

Get glowing in two easy steps: step 1: cleanse with love your bare face hydrating cleansing oil. Pump a liberal amount into dry hands and massage over dry face to instantly dissolve makeup, dirt, and impurities. Enriched with a blend of grape seed oil that soothes skin and tightens pores, grapefruit peel oil cleanses and evens skin tone, rosehip seed oil hydrates and restores ph balance, and green coffee bean oil invigorates and promotes cell turnover. In a consumer use study, 91 percent of participants agreed bare face cleansing oil leaves their skin softer; 94 percent reported there was no oily residue after using bare face; and 97 percent agreed bare face cleansing oil did not irritate skin. Step 2: exfoliate with charcoal konjac sponge. Dampen sponge and rub gently in a circular motion to slough, smooth, and clean. Use warm water to rinse face and wash away cleansing oil. Made for daily use. Konjac root gently exfoliates and stimulates blood circulation while charcoal powder detoxi

In [141]:
for _ in all_review_list[all_review_list['asin']==diffcat_conc_asin]['topic_3_sent']:
    print(_)

['Juelp provides the best skin care regimen for my sensitive skin.', 'I love it, I love the Kojac sponge effective exfoliating without damaging my skin!The Konjac sponge is gentle enough to use everyday (I have sensitive and acne prone skin).']


In [160]:
recs.iloc[0:5][['other_asin','product','topic_6_sent','topic_6_score','new_sim']]

,other_asin,product,topic_6_sent,topic_6_score,new_sim
35,B004WDLXI4,"St. Tropez Skin Illuminator, Gold, 1.6 fl. oz.",[Please give it try it will take your look nat...,0.9124,0.865542
111,B00LLJI36C,TNS Recovery Complex,[GreatJust re-purchased... Love this entire li...,0.9199,0.840827
62,B00DQYSXCY,AHAVA Dead Sea Osmoter Concentrate,[It's just helpful to have something to slow i...,0.8885,0.831469
106,B0032Z24M2,"Natura Bisse The Cure Pure Serum, 1.0 fl. oz.",[My skin has looked so good since switching to...,0.8788,0.804085
10,B000M4Z3OW,"NEOVA Serious Clarity Microderm Scrub, 2.5 Fl Oz",[Helps a great deal with fine lines.],0.8176,0.795044


In [152]:
all_review_list[all_review_list['asin']=='B005UPIVOC']

,asin,title,all_review_text,sentences,compound_scores,pos_sents,pos_scores,neg_sents,neg_scores,topic_3_sent,topic_3_score,topic_4_sent,topic_4_score,topic_5_sent,topic_5_score,topic_6_sent,topic_6_score
982,B005UPIVOC,Indie Lee Blemish Stick,As a mom and aesthetician I have tried endless...,[As a mom and aesthetician I have tried endles...,"[0.6696, 0.0, 0.6249, 0.0, 0.0, 0.0, 0.125, 0....",[As a mom and aesthetician I have tried endles...,"[0.6696, 0.6249, 0.125, 0.9532, 0.4389, 0.1779...","[Works overnight to reduce breakouts., At firs...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.3089, -0.619...",[],NaN,[As a 40 year old woman who has battled acne s...,0.373067,"[Tea tree oil seemed to work better,]",0.4404,[:) My body naturally rejects anything that tr...,0.9532


In [151]:
for _ in all_review_list[all_review_list['asin']=='B005UPIVOC']['pos_sents']:
    print(_)

['As a mom and aesthetician I have tried endless products and this one is the best!', 'This is a great product.', "It's not as good as reviews I've read has said, but it's pretty good.", ":) My body naturally rejects anything that tries to help it, so I'm surprised this worked at least a little haha Amazing customer service and a great line of products :)Its literally alcohol, for the money use Q-tip and own alcohol save yourself few bucks.", 'As a 40 year old woman who has battled acne since I was a teenager, I think this stuff is a miracle!', 'It will easily dry out my skin now that Im older if I use more than just a small amount, but that is enough to dry up any blemishes.', 'My two teenage daughters who have my skin can tolerate more of it and it is very effective.', 'Perfect for suddenly appearing blemishes.', 'Works overnight!I really like this product.', '!I love this product so much!', 'I have tried virtually everything to help alleviate my breakouts: clay masks, charcoal masks

### find a breakout product 

In [109]:
a1 = all_review_list.sort_values('topic_4_score',ascending=False)
a1.iloc[0:15][['asin','title','topic_4_score']]

,asin,title,topic_4_score
2073,B019QSK1YG,"Julep Korean Skincare Exfoliating, Hydrating F...",0.95450
389,B00172JYKW,"Mario Badescu Cellufirm Moisturizer, 2 oz.",0.94970
58,B00070JJ4G,"NEOVA Reveal Exfoliator, 20% Glycolic, 2.0 Fl Oz",0.94460
986,B005WOIV7S,PMD Soothing Antioxidant Cleanser,0.94410
1542,B00JH8OGH4,MEG 21 Clean and Clear Moisturizing Cleanser C...,0.94220
1233,B00BVO6WN8,"Hydrance LIGHT Hydrating Emulsion, 1.35 fl. oz",0.93980
1437,B00GPQ9FQI,bliss labs Active 99.0 Anti-Aging Series Refin...,0.91680
909,B0058EELK8,Jan Marini Skin Research Benzoyl Peroxide Acne...,0.90420
1777,B00QRDFTH8,TONYMOLY Magic Food Banana Sleeping Pack,0.90065
1546,B00JJ4OCR0,Anthony Logistics for Men Purifying Astringent...,0.89790


In [221]:
a = max(all_review_list['topic_4_score'])

In [222]:
all_review_list.loc[all_review_list['topic_4_score']==a]

,asin,title,all_review_text,sentences,compound_scores,pos_sents,pos_scores,neg_sents,neg_scores,topic_3_sent,topic_3_score,topic_4_sent,topic_4_score,topic_5_sent,topic_5_score,topic_6_sent,topic_6_score
2073,B019QSK1YG,"Julep Korean Skincare Exfoliating, Hydrating F...",Can really see the difference with my skin... ...,[Can really see the difference with my skin......,"[0.0, 0.0, 0.6817, 0.1779, 0.128, 0.0, 0.0, 0....","[One of my FAVORITE Julep products!!!, This is...","[0.6817, 0.1779, 0.128, 0.128, 0.7959, 0.4574,...",[Can really see the difference with my skin......,"[0.0, 0.0, 0.0, 0.0, -0.2057, 0.0, -0.8251, 0....",[Juelp provides the best skin care regimen for...,0.88355,"[I love it, I love the Kojac sponge effective ...",0.9545,[This cleansing oil leaves my skin feeling so ...,0.580058,[],NaN


### repeat for acne example

In [100]:
main_prod3, recs3 = get_rec('B00JH8OGH4','none')
main_prod4, recs4 = get_rec('B00JH8OGH4','acne_control')

In [101]:
main_prod3[['asin','product','category2','overall_rating','description']]

,asin,product,category2,overall_rating,description
1542,B00JH8OGH4,MEG 21 Clean and Clear Moisturizing Cleanser C...,Cleansers,3.6,"This scientifically formulated, refreshing and..."


In [102]:
recs3[['other_asin','product','category2','overall_rating','similarity','new_sim','topic_4_score']]

,other_asin,product,category2,overall_rating,similarity,new_sim,topic_4_score
34,B00C1SZHBG,"DR. ALKAITIS Organic Flower Mask, 1.25 fl. oz.",Treatments & Masks,5.0,0.926283,0.926,NaN
130,B000CCML94,DERMAdoctor Wrinkle Revenge 1 antioxidant enha...,Treatments & Masks,4.3,0.870996,0.871,0.480743
163,B0177RLDYU,"THALGO Mceutic Pro-Renewal Cleanser, 6.76 oz",Treatments & Masks,5.0,0.855776,0.856,0.802000
189,B002RCQJ2W,Red Flower Lymphatic Phytopower Sea Cleanser a...,Treatments & Masks,4.4,0.841631,0.842,0.176100
201,B00HYLOVLW,"Pevonia Micro Retinol Foaming Cleanser, 4 Fl Oz",Treatments & Masks,3.8,0.828094,0.828,0.648600
56,B01AHOFREW,jane iredale BeautyPrep Face Toner,Toners & Astringents,3.5,0.913357,0.913,NaN
117,B0019V5HI4,"Pevonia Combination Skin Cleanser, 6.8 Fl Oz",Toners & Astringents,3.6,0.876647,0.877,0.486833
118,B0018QXRCI,"Elemis Cleanser, Rehydrating Ginseng Toner For...",Toners & Astringents,5.0,0.875855,0.876,NaN
216,B00H59AA2W,"Guinot Hydra Beaute Toning Lotion, 6.7 Fl oz",Toners & Astringents,4.8,0.816694,0.817,NaN
231,B00T72SWZM,"COSMEDIX Purity Solution, 3.3 Fluid Ounce",Toners & Astringents,5.0,0.797172,0.797,0.438000


In [103]:
recs4[['other_asin','product','category2','overall_rating','similarity','new_sim','topic_4_score']]

,other_asin,product,category2,overall_rating,similarity,new_sim,topic_4_score
163,B0177RLDYU,"THALGO Mceutic Pro-Renewal Cleanser, 6.76 oz",Treatments & Masks,5.0,0.855776,0.686,0.802000
201,B00HYLOVLW,"Pevonia Micro Retinol Foaming Cleanser, 4 Fl Oz",Treatments & Masks,3.8,0.828094,0.537,0.648600
285,B001AIMZCW,"Elemis Maximum Replenish Night Cream, 1.7 Ounce",Treatments & Masks,5.0,0.718976,0.514,0.715500
666,B0058EELK8,Jan Marini Skin Research Benzoyl Peroxide Acne...,Treatments & Masks,3.6,0.465219,0.421,0.904200
130,B000CCML94,DERMAdoctor Wrinkle Revenge 1 antioxidant enha...,Treatments & Masks,4.3,0.870996,0.419,0.480743
464,B00JJ4OCR0,Anthony Logistics for Men Purifying Astringent...,Toners & Astringents,4.0,0.521911,0.469,0.897900
117,B0019V5HI4,"Pevonia Combination Skin Cleanser, 6.8 Fl Oz",Toners & Astringents,3.6,0.876647,0.427,0.486833
297,B002Q7K820,"Pevonia Spateen Blemished Skin Toner, 4 Fl Oz",Toners & Astringents,4.3,0.691276,0.416,0.601700
388,B000RTTVGS,"Guinot Microbiotic Toning Lotion, 6.7 fl.oz.",Toners & Astringents,4.6,0.586300,0.366,0.624900
231,B00T72SWZM,"COSMEDIX Purity Solution, 3.3 Fluid Ounce",Toners & Astringents,5.0,0.797172,0.349,0.438000


#### compare top rated in that category for each

In [104]:
#no concerns - 
samecat_none_asin = 'B00GTESTLS'
for _ in recs3.loc[recs3['other_asin']==samecat_none_asin]['description']:
    print(_)

Purify, soften and balance skins natural moisture with this deep cleansing oil formulated with grapeseed, may chang, ho wood and sunflower oils. A proprietary blend of fruit stem cells and vitamin C are infused into an organic, resveratrol-rich grapeseed base while organic fruit acids brighten skin tone and texture and help to dissolve impurities. Ideal for all skin types (especially for skin showing signs of aging, fine lines, and wrinkles). Non-petroleum based oil products can balance oil without clogging pores or drying out your skin. A tried and true method with origins dating back to Asia as far back as the fourteenth century. As a nightly skin care routine, you use a cleansing oil first to remove any makeup and then a nightly facial cleanser to clean and hydrate your skin.


In [105]:
for _ in all_review_list[all_review_list['asin']==samecat_none_asin]['topic_4_sent']:
    print(_)

["I've never used a cleansing oil because I hate anything thick and greasy on my face but I don't have that problem with this cleanser because it leaves my face leaving so amazingly soft and looking so clean.", 'Love this product line so much!I have oily, acne prone skin.', "I was nervous at first to buy this because I have really oily & acne prone skin & was nervous to wash my face with oil but I surprisingly haven't broke out once from it!", 'I bought this as a makeup remover because oil are much more gentle on my skin and I am less prone to acne breakouts.', 'The apricot kernel oil smells great, and has kept my skin feeling soft, smooth, and breakout free.', "Others may have no problem with this but since I use so little on my face it doesn't take much to make me feel like I do have a coating -- so take that into consideration because on the whole I really like the way the cleansing oil make my skin feel -- soft and pliant.", 'None of these problems exist with Juice Beauty Cleansing

In [107]:
#acne - very positive sentiment
samecat_conc_asin = 'B005WOIV7S'
for _ in recs4.loc[recs4['other_asin']==samecat_conc_asin]['description']:
    print(_)

Begin your PMD regimen with a fresh, purified face. PMD Advanced Soothing Cleanser deeply cleans down to the pores, removing dirt and oil without damaging the skin's natural barrier. The ultra-gentle formula soothes and reduces redness and irritation, aiding the skin's natural healing and recovery process.


In [108]:
for _ in all_review_list[all_review_list['asin']==samecat_conc_asin]['topic_4_sent']:
    print(_)

['Love love love!The product is amazing does so much to keep my face clear of pimplesI can really tell the difference using this cleanser.']


#### compare top rated in other categories

In [28]:
#no concerns - mask
# diffcat_none_asin = 'B008H6MSZO' #mask
# diffcat_none_asin = 'B00260GSWQ' #exfoliator
diffcat_none_asin = 'B01HBS7XP8' #toner
for _ in recs3.loc[recs3['other_asin']==diffcat_none_asin]['description']:
    print(_)

Soothe, calm and nourish the skin with green tea, aloe vera and chamomile after a deep cleanse with Benefit Balance. Providing a second clean sweep of the skin with lactic acid and witch hazel, this antioxidant-infused toner helps protect the skin from harmful free radicals. D-glucuronic acid and shea butter help balance the skin and prevent moisture loss, making the skin perfectly prepped for serums or moisturizers. COSMEDIX skincare is formulated with the cleanest, most potent plant-based ingredients, Nobel Prize-winning chemistry and gentle actives to deliver safe, effective results. Trusted by dermatologists and aestheticians worldwide. With super clean ingredients, luxurious textures and pure botanicals, COSMEDIX gently delivers beautiful, transformative results for all skin types and tones. Our formulas feature higher concentrations of active ingredients and are created with a focus on crafting effective, gentle products that transform your skin naturally. Soothe, calm and nouris

In [29]:
for _ in all_review_list[all_review_list['asin']==diffcat_none_asin]['topic_3_sent']:
    print(_)

[]


In [30]:
#sensitive skin - 
# diffcat_conc_asin = 'B002RCQJ2W' #mask
# diffcat_conc_asin = 'B019QSK1YG' #exfoliator
diffcat_conc_asin = 'B002Q7K820' #toner
for _ in recs4.loc[recs4['other_asin']==diffcat_conc_asin]['description']:
    print(_)

Remove impurities and excess oil that can clog pores. Complete the cleanse, remove tap water impurities, control bacteria, soothe, heal and prepare the skin for further treatment with this alcohol-free toner. For teens and adults with acne and skin that tends to be sensitive or easily gets red and irritated. Favored by influencers, skincare experts and elite spas worldwide, award-winning Pevonia is clinically proven to work. Sustainably sourced, products are rich in potent organic ingredients for immediate and long-lasting results.


In [31]:
for _ in all_review_list[all_review_list['asin']==diffcat_conc_asin]['topic_3_sent']:
    print(_)

['It is a very gentle cleanser so i highly recommend it for those with sensitive skin.']


### Pull example for positive acne sentiment examples

In [174]:
a1 = all_review_list.sort_values('topic_4_score',ascending=False)
a1.iloc[85:105][['asin','title','topic_4_score','topic_4_score1']]

,asin,title,topic_4_score,topic_4_score1
472,B001DYDFMC,"Mario Badescu Cream X, 1 oz.",0.718400,[0.7184]
1779,B00QPNKVTQ,"Erno Laszlo Marble Treatment Bar, White, 5.3 oz.",0.718400,[0.7184]
223,B000RZVK02,AHAVA Sun Protection Anti-Aging Moisturizer,0.718400,[0.7184]
1476,B00HWYO760,Clearogen Acne Lotion Benzoyl Peroxide for acn...,0.715775,"[0.7359, 0.8065, 0.3612, 0.9595]"
448,B001AIMZCW,"Elemis Maximum Replenish Night Cream, 1.7 Ounce",0.715500,[0.7155]
1614,B00L3EORUG,"Purifying Cleanser, Anti Wrinkle + Clarify",0.712500,[0.7125]
2093,B01AZR97RE,BA2:B42ioderma Hydrabio Moisturizing Face Crea...,0.709600,[0.7096]
237,B000UFPC9Y,Epicuren Discovery Clarify Cleanser,0.709140,"[0.7615, 0.6249, 0.6486, 0.6588, 0.8519]"
1025,B006ZTZZDW,"Rodial Dragon's Blood Sculpting Gel, 1.7 oz",0.708900,[0.7089]
741,B003BOK7BY,DDF Weightless Defense Oil-Free Hydrator UV Mo...,0.706050,"[0.8402, 0.5719]"


In [185]:
#B000ULLISW, B000R8Z852, B000UFPC9Y
asin1 = 'B00HWYO760' #use for acne
# asin1 = 'B000ULLISW'
for _ in all_review_list[all_review_list['asin']==asin1]['topic_4_sent']:
    print(_)
for _ in all_review_list[all_review_list['asin']==asin1]['topic_4_score1']:
    print(_)

['My son has tried multiple skin care lines for his acne, this was the first that REALLY cleared up his skin!!', '!This truly is a miracle for acne!', 'It even helped with rosacea like breakouts I used to get on my cheeks and forehead during hormonal changes too.', 'So happy that I found this natural ingredients Amazon find!Works great for meI LOVE the Clearogen acne lotion, I tend to get random breakouts on my face since using this lotion, my skin is glowing!']
[0.7359, 0.8065, 0.3612, 0.9595]


[0.7359, 0.8065, 0.3612, 0.9595]


In [181]:
product_list[product_list['asin']==asin1]

,id,asin,product,overall_rating,review_count,description,category2,category3
1476,5dc881789f9b98109203da2e,B00HWYO760,Clearogen Acne Lotion Benzoyl Peroxide for acn...,4.2,6,Clearogen Benzoyl Peroxide Acne Lotion is a p...,Cleansers,Not listed


In [ ]:
# Save out dictionaries for flask app
# dictionary of product info - asin, product name, description, overall rating, category, topic values?
# dictionary of similarities (sim2)